In [1]:
import torch
import numpy as np
import random
import sentencepiece as spm
from transformers import ReformerConfig, ReformerTokenizer, ReformerModel, PreTrainedModel
from torch.utils.data import DataLoader, Dataset


NUM_BATCHES = None
BATCH_SIZE = 6
GRADIENT_ACCUMULATE_EVERY = 3
LEARNING_RATE = 0.01 #1e-4
VALIDATE_EVERY  = 12
SEQ_LEN = 4608 # 4608/64=72, 4608=64*72

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [3]:
# def split_file(file,out1,out2,percentage=0.75,isShuffle=True,seed=42):
#     """quora.com/How-can-split-a-text-file-randomly-in-75-and-25-and-create-two-output-file-in-python
#     """
#     random.seed(seed)
#     with open(file, 'r',encoding="utf-8") as fin, open(out1, 'w') as foutBig, open(out2, 'w') as foutSmall:
#         nLines = sum(1 for line in fin)
#         fin.seek(0)

#         nTrain = int(nLines*percentage) 
#         nValid = nLines - nTrain

#         i = 0
#         for line in fin:
#             r = random.random() if isShuffle else 0 # so that always evaluated to true when not isShuffle
#             if (i < nTrain and r < percentage) or (nLines - i > nValid):
#                 foutBig.write(line)
#                 i += 1
#             else:
#                 foutSmall.write(line)
                
# split_file("data/yeast/yeast.txt", 
#            "data/yeast/yeast_train.txt",
#            "data/yeast/yeast_val.txt",
#            percentage=0.9)

In [3]:
spm.SentencePieceTrainer.Train("--input=./data/tokenizer_training/AAresiduals.txt \
                                --vocab_size=28 \
                                --model_prefix=sequence_tokenizer \
                                --model_type=char \
                                --character_coverage=1.0")
tokenizer = ReformerTokenizer(vocab_file="sequence_tokenizer.model", do_lower_case=False, model_max_length=SEQ_LEN)

In [96]:
# tokenizer.save_vocabulary('./')

In [97]:
tokenizer.all_special_tokens, tokenizer.all_special_ids, tokenizer.vocab_size

(['<unk>', '<pad>', '</s>'], [0, 0, 2], 28)

In [104]:
[(v, tokenizer.decode(v)) for i, v in enumerate(range(28))]

[(0, '<unk>'),
 (1, '<s>'),
 (2, '</s>'),
 (3, 'A'),
 (4, 'B'),
 (5, 'C'),
 (6, 'D'),
 (7, 'E'),
 (8, 'F'),
 (9, 'G'),
 (10, 'H'),
 (11, 'I'),
 (12, 'K'),
 (13, 'L'),
 (14, 'M'),
 (15, 'N'),
 (16, 'O'),
 (17, 'P'),
 (18, 'Q'),
 (19, 'R'),
 (20, 'S'),
 (21, 'T'),
 (22, 'U'),
 (23, 'V'),
 (24, 'W'),
 (25, 'X'),
 (26, 'Y'),
 (27, 'Z')]

In [117]:
print(tokenizer.bos_token, tokenizer.bos_token_id, tokenizer.eos_token, tokenizer.eos_token_id,
tokenizer.unk_token, tokenizer.unk_token_id, tokenizer.pad_token, tokenizer.pad_token_id,
tokenizer.mask_token, tokenizer.mask_token_id)

Using mask_token, but it is not set yet.
Using mask_token, but it is not set yet.


<s> 1 </s> 2 <unk> 0 <pad> 0 None None


In [118]:
# tokenizer._bos_token = '<s>'
# # tokenizer._bos_token_id = 1
# tokenizer._eos_token = '</s>'
# # tokenizer._eos_token_id = 2
# tokenizer._unk_token = '<unk>'
# # tokenizer._unk_token_id = 3
# tokenizer._pad_token = '<pad>'
# # tokenizer._pad_token_id = 0
# tokenizer._mask_token = '<mask>'
# # tokenizer._mask_token_id = 4


In [119]:
tokenizer.all_special_ids

[2, 1, 0, 0, 0]

In [120]:
tokenizer.all_special_tokens

['</s>', '<s>', '<mask>', '<unk>', '<pad>']

In [124]:
tokenizer.batch_encode_plus(['AB<mask>CD', 
                             'BBCD', 
                             'CBCD'], add_special_tokens=True, pad_to_max_length=True, max_length=10)

{'input_ids': [[0, 3, 4, 0, 5, 6, 0, 0, 0, 0], [0, 4, 4, 5, 6, 0, 0, 0, 0, 0], [0, 5, 4, 5, 6, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}

In [7]:
# with open('data/yeast/yeast.txt') as file:
#     X = [l.strip() for l in file]
#     y = [len(l) for l in X]
# print(max(y))

In [8]:
class SequenceDataset(Dataset):
    def __init__(self, masked_input_ids, masked_labels, attention_mask, tokenizer, _len):
        super().__init__()
        self.input_ids = masked_input_ids
        self.labels = masked_labels
        self.attention_mask = attention_mask
        self.tokenizer = tokenizer
        self._len = _len

    @classmethod
    def prepare_from_file(cls, file_path, tokenizer):
        with open(file_path) as file:
            X = [l.strip() for l in file]
            # cut sequence into certain length (e.g. if max length is 10, cut to 10)
            X = [tokenizer.encode(sequence)[1:tokenizer.max_len+1] for sequence in X]
            temp = [cls.mask_random(sequence) for sequence in X]
            masked_sequences = [t[0] for t in temp]
            masked_labels = [t[1] for t in temp]
            temp = [tokenizer.prepare_for_model(masked_sequence, 
                                                max_length=tokenizer.max_len,
                                                pad_to_max_length=True) for masked_sequence in masked_sequences]
            masked_labels = [tokenizer.prepare_for_model(masked_label,
                                                         max_length=tokenizer.max_len,
                                                         pad_to_max_length=True)["input_ids"] for masked_label in masked_labels]
            masked_input_ids = [torch.tensor(l["input_ids"]) for l in temp]
            masked_labels = [torch.tensor(masked_label) for masked_label in masked_labels]
            attention_mask = [torch.tensor(l["attention_mask"]) for l in temp]
            del(temp)
        masked_input_ids = torch.stack([masked_input_ids[i] for i in range(len(masked_input_ids))]).squeeze()
        masked_labels = torch.stack([masked_labels[i] for i in range(len(masked_labels))]).squeeze()
        attention_mask = torch.stack([attention_mask[i] for i in range(len(attention_mask))]).squeeze()
        
        return cls(masked_input_ids, masked_labels, attention_mask, tokenizer, len(masked_input_ids))
    
    @staticmethod
    def mask_random(sequence):
        masked_sequence = [residual for residual in sequence]
        masked_labels = []
        
        for i, residual in enumerate(masked_sequence):
            prob = random.random()
            if prob < 0.15:
                prob /= 0.15
                if prob < 0.8:
                    masked_sequence[i] = 0
                elif prob < 0.9:
                    masked_sequence[i] = random.randrange(tokenizer.vocab_size)
                masked_labels.append(residual)
            else:
#                 masked_labels.append(-100)
                masked_labels.append(0)
        return masked_sequence, masked_labels

    def __getitem__(self, index):
        return self.input_ids[index, ].cuda(), self.labels[index, ].cuda(), self.attention_mask[index, ].cuda()

    def __len__(self):
        return self._len

In [9]:
# x = SequenceDataset.prepare_from_file("data/yeast/yeast.txt", tokenizer)

In [10]:
# x.labels, x.attention_mask, x.input_ids, x.labels.shape, x.attention_mask.shape, x.input_ids.shape

In [11]:
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from transformers.modeling_reformer import ACT2FN, ReformerModel, PreTrainedModel, ReformerPreTrainedModel
ReformerLayerNorm = torch.nn.LayerNorm
ACT2FN = {"relu": torch.nn.functional.relu}


class ReformerMLMHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.dense = nn.Linear(config.hidden_size * 2, config.hidden_size)
        if isinstance(config.hidden_act, str):
            self.transform_act_fn = ACT2FN[config.hidden_act]
        else:
            self.transform_act_fn = config.hidden_act
        self.LayerNorm = ReformerLayerNorm(config.hidden_size, eps=config.layer_norm_eps)

        self.decoder = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
#         print(self.decoder)
        self.bias = nn.Parameter(torch.zeros(config.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        logits = self.decoder(hidden_states)
        return logits


class ReformerMLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.reformer = ReformerModel(config)
        self.mlm = ReformerMLMHead(config)
        
    def forward(self, input_ids=None, attention_mask=None, position_ids=None, head_mask=None, 
                inputs_embeds=None, labels=None, output_attentions=None, **kwargs):
        reformer_outputs = self.reformer(input_ids, position_ids=position_ids, attention_mask=attention_mask,
                                          head_mask=head_mask, inputs_embeds=inputs_embeds, num_hashes=None)

        hidden_states = reformer_outputs[0]
#         print("output shape of the ReformerModel: {}".format(hidden_states.shape))
        logits = self.mlm(hidden_states)
#         print("logits shape: {}".format(logits.shape))
        outputs = (logits,) + reformer_outputs[1:]

        if labels is not None:
            loss_fct = CrossEntropyLoss(ignore_index=0)  # -100 index = padding token
            masked_lm_loss = loss_fct(logits.view(-1, self.config.vocab_size), labels.view(-1))
            outputs = (masked_lm_loss,) + outputs
        return outputs  # (lm_loss), lm_logits, (hidden_states), (attentions)
    


In [12]:
tokenizer.vocab_size

28

In [13]:
configuration = ReformerConfig()
configuration.axial_pos_shape = (64, 72)
configuration.max_position_embeddings = SEQ_LEN # 4608
configuration.vocab_size = tokenizer.vocab_size # 28
configuration.architectures=None
configuration.is_decoder=False
configuration.hidden_size=256 # ReformerModel output size = 512
configuration.save_pretrained('model/fix_masked_LM_config/')
configuration = ReformerConfig.from_pretrained('model/fix_masked_LM_config/')

model = ReformerMLM(configuration)

In [14]:
# lab_x=torch.zeros((2,4608), dtype=torch.int64)

In [15]:
# lab_x

In [16]:
# outputs = model(input_ids=inp, attention_mask=att, labels=lab)
# When training a model from scratch, it is recommended to leave ``config.num_buckets=None``, 
# so that depending on the sequence length a good value for ``num_buckets`` is calculated on the fly. 
# This value will then automatically be saved in the config and should be reused for inference.

In [17]:
train_dataset = SequenceDataset.prepare_from_file("data/yeast/yeast_train.txt", tokenizer)
val_dataset = SequenceDataset.prepare_from_file("data/yeast/yeast_val.txt", tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (4911 > 4608). Running this sequence through the model will result in indexing errors


In [18]:
# train_dataset.labels

In [19]:
def cycle(loader):
    while True:
        for data in loader:
            yield data
            
def collate_fn(batch):
    """
    batch:[(inputs, labels, mask),...]
    [(tensor([14,  3, 11, 0,  0], device='cuda:0'), 
      tensor([0, 0, 0, 0, 0], device='cuda:0'), 
      tensor([1, 1, 1, 0, 0], device='cuda:0')), ...]
    """
    inputs = torch.stack([x[0] for x in batch])
    labels = torch.stack([x[1] for x in batch])
    attention_mask = torch.stack([x[2] for x in batch])

    output = {"inputs": inputs,
              "labels": labels,
              "attention_mask": attention_mask}

    return output

train_loader = cycle(DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE))
val_loader = cycle(DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE))

In [20]:
next(train_loader)

{'inputs': tensor([[14,  0, 11,  ...,  0,  0,  0],
         [14, 26, 26,  ...,  0,  0,  0],
         [14, 20,  3,  ...,  0,  0,  0],
         [14, 15,  0,  ...,  0,  0,  0],
         [14, 10, 21,  ...,  0,  0,  0],
         [14, 11,  0,  ...,  0,  0,  0]], device='cuda:0'),
 'labels': tensor([[ 0,  3,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  7,  ...,  0,  0,  0],
         [ 0,  0, 21,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0, 19,  ...,  0,  0,  0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [21]:
from transformers import AdamW
optimizer = AdamW(params=model.parameters(), lr=LEARNING_RATE)
NUM_BATCHES = len(train_dataset)//BATCH_SIZE

model.cuda()

ReformerMLM(
  (reformer): ReformerModel(
    (embeddings): ReformerEmbeddings(
      (word_embeddings): Embedding(28, 256)
      (position_embeddings): AxialPositionEmbeddings(
        (weights): ParameterList(
            (0): Parameter containing: [torch.cuda.FloatTensor of size 64x1x64 (GPU 0)]
            (1): Parameter containing: [torch.cuda.FloatTensor of size 1x72x192 (GPU 0)]
        )
      )
    )
    (encoder): ReformerEncoder(
      (layers): ModuleList(
        (0): ReformerLayer(
          (attention): ReformerAttention(
            (layer_norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
            (self_attention): LocalSelfAttention(
              (query): Linear(in_features=256, out_features=128, bias=False)
              (key): Linear(in_features=256, out_features=128, bias=False)
              (value): Linear(in_features=256, out_features=128, bias=False)
            )
            (output): ReformerSelfOutput(
              (dense): Linear(in_features=

In [23]:
import tqdm
for x in range(5):
    for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):

        model.train()

        for __ in range(GRADIENT_ACCUMULATE_EVERY):
            tmp = next(train_loader)
            inputs = tmp['inputs']
            labels = tmp['labels']
            attention_mask = tmp['attention_mask']

            outputs = model(input_ids=inputs, labels=labels, attention_mask=attention_mask)
            loss, prediction_scores = outputs[:2]
            loss.backward()

        print(f'training loss: {loss.item()}')

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()
        optimizer.zero_grad()

        if i % VALIDATE_EVERY == 0:
            model.eval()
            with torch.no_grad():
                tmp = next(train_loader)
                inputs = tmp['inputs']
                labels = tmp['labels']
                attention_mask = tmp['attention_mask']
                outputs = model(input_ids=inputs, labels=labels, attention_mask=attention_mask)
                loss, prediction_scores = outputs[:2]
                print(f'validation loss: {loss.item()}')

training:   0%|                                                                                                                                  | 0/907 [00:00<?, ?it/s]

training loss: 2.9347362518310547
validation loss: 2.8914644718170166
training loss: 2.8864529132843018
training loss: 2.9085187911987305
training loss: 2.9765799045562744
training loss: 2.9061970710754395
training loss: 2.887698173522949
training loss: 2.9558684825897217
training loss: 2.8694169521331787
training loss: 2.8877532482147217
training loss: 2.901780843734741
training loss: 2.9264981746673584
training loss: 2.887889862060547
training loss: 2.864699602127075
validation loss: 2.9043123722076416
training loss: 2.8842380046844482


training:   2%|██                                                                                                                       | 15/907 [00:10<10:35,  1.40it/s]

training loss: 2.868661880493164
training loss: 2.8706462383270264
training loss: 2.8996357917785645
training loss: 2.8958234786987305
training loss: 2.864328622817993
training loss: 2.8696482181549072
training loss: 2.8836920261383057
training loss: 2.8888814449310303
training loss: 2.9204964637756348
training loss: 2.9000773429870605
training loss: 2.880237579345703
validation loss: 2.902726888656616
training loss: 2.865835666656494
training loss: 2.8833723068237305
training loss: 2.9310190677642822
training loss: 2.8786799907684326


training:   3%|████                                                                                                                     | 30/907 [00:21<10:23,  1.41it/s]

...

validation loss: 2.8553528785705566
training loss: 2.965726852416992
training loss: 2.9269609451293945
training loss: 2.8579893112182617


training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 905/907 [57:04<00:06,  3.31s/it]

training loss: 2.909682273864746
training loss: 2.860117197036743


training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 907/907 [57:09<00:00,  3.78s/it]

training loss: 2.9044148921966553


In [12]:
import torch
x=torch.rand((3,2,5))
x

tensor([[[0.4644, 0.6206, 0.9698, 0.7029, 0.1203],
         [0.3409, 0.8936, 0.4948, 0.7733, 0.4185]],

        [[0.0241, 0.5471, 0.6334, 0.2244, 0.3367],
         [0.4312, 0.7499, 0.9411, 0.6488, 0.4058]],

        [[0.3744, 0.3766, 0.0417, 0.9009, 0.0848],
         [0.0123, 0.6774, 0.8277, 0.1902, 0.2619]]])

In [13]:
x[..., :-1, :]

tensor([[[0.4644, 0.6206, 0.9698, 0.7029, 0.1203]],

        [[0.0241, 0.5471, 0.6334, 0.2244, 0.3367]],

        [[0.3744, 0.3766, 0.0417, 0.9009, 0.0848]]])

In [14]:
x[...,1:]#.reshape(-1)

tensor([[[0.6206, 0.9698, 0.7029, 0.1203],
         [0.8936, 0.4948, 0.7733, 0.4185]],

        [[0.5471, 0.6334, 0.2244, 0.3367],
         [0.7499, 0.9411, 0.6488, 0.4058]],

        [[0.3766, 0.0417, 0.9009, 0.0848],
         [0.6774, 0.8277, 0.1902, 0.2619]]])